In [1]:
import pandas as pd
import numpy as np

### Cleaning data
#### Step by step note about how I have cleaned this data.
* Rename all columns and make column __Salary Estimate__ more readible,
* Creating __Max_Salary__ and __Min_Salary__ by spliting column __Salary Estimate__ and working with these columns,
* Droping duplicates,
* Clean and split column __Location__,
* Check __Index__ for values beside float,
* Column __Rating__ looking for different values and fix values that are lower than 1 to 0,
* __Company_Name__ column, cleaning rightside of the every row that has unnecassary information ,
* Cleaning column __Headquarters__,
* Cleaning column __Size__,
* Cleaning column __Type of ownership__,
* Cleaning column __Industry__,
* Cleaning column __Sector__, 
* Cleaning column __Revenue__,
* Cleaning column __Competitors__,
* Deleting __Salary Estimate__, __Location__ and __index__ columns,
* Cleaning column __Max_Salary__,
* Creating a column which will have job name (data analyst, data engineer or data scientist) if this job name is in __Job_Title__,
* Creating column __seniority__,
* Creating columns for every key skill if it is in column __Job_Description__,
* New order for columns,
* Write clean and modified data to new csv file

In [2]:
df = pd.read_csv(r'Uncleaned_DS_jobs.csv')
df.head(10) 

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,1,Data Scientist,$137K-$171K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,2,Data Scientist,$137K-$171K (Glassdoor est.),Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$137K-$171K (Glassdoor est.),JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,4,Data Scientist,$137K-$171K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"
5,5,Data Scientist,$137K-$171K (Glassdoor est.),About Us:\n\nHeadquartered in beautiful Santa ...,4.2,HG Insights\n4.2,"Santa Barbara, CA","Santa Barbara, CA",51 to 200 employees,2010,Company - Private,Computer Hardware & Software,Information Technology,Unknown / Non-Applicable,-1
6,6,Data Scientist / Machine Learning Expert,$137K-$171K (Glassdoor est.),Posting Title\nData Scientist / Machine Learni...,3.9,Novartis\n3.9,"Cambridge, MA","Basel, Switzerland",10000+ employees,1996,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$10+ billion (USD),-1
7,7,Data Scientist,$137K-$171K (Glassdoor est.),Introduction\n\nHave you always wanted to run ...,3.5,iRobot\n3.5,"Bedford, MA","Bedford, MA",1001 to 5000 employees,1990,Company - Public,Consumer Electronics & Appliances Stores,Retail,$1 to $2 billion (USD),-1
8,8,Staff Data Scientist - Analytics,$137K-$171K (Glassdoor est.),Intuit is seeking a Staff Data Scientist to co...,4.4,Intuit - Data\n4.4,"San Diego, CA","Mountain View, CA",5001 to 10000 employees,1983,Company - Public,Computer Hardware & Software,Information Technology,$2 to $5 billion (USD),"Square, PayPal, H&R Block"
9,9,Data Scientist,$137K-$171K (Glassdoor est.),Ready to write the best chapter of your career...,3.6,XSELL Technologies\n3.6,"Chicago, IL","Chicago, IL",51 to 200 employees,2014,Company - Private,Enterprise Software & Network Solutions,Information Technology,Unknown / Non-Applicable,-1


#### Rename all columns and make column __Salary Estimate__ more readible

In [3]:
df = df.rename(columns= {'Job Description': 'Job_Description','Job Title': 'Job_Title', 'Company Name': 'Company_Name', 'Type of ownership': 'Type_of_ownership'})

In [4]:
df['Salary Estimate'] = df['Salary Estimate'].str.replace('K', '000')
df['Salary Estimate'] = df['Salary Estimate'].str.replace('$', '')


#### Creating __Max_Salary__ and __Min_Salary__ by spliting column __Salary Estimate__ and working with these columns.

In [5]:
df[['Min_Salary', 'Max_Salary']] = df["Salary Estimate"].str.split('-', expand=True)

In [6]:
filt = df['Max_Salary'].str.contains('Employer est.', regex=False)
df.loc[filt, 'Max_Salary'] = df.loc[filt, 'Max_Salary'].str[:6]

In [7]:
filt = df['Max_Salary'].str.contains('Glassdoor est.', regex=False)
df.loc[filt, 'Max_Salary'] = df.loc[filt, 'Max_Salary'].str[:-17]

In [8]:
df[['Min_Salary', 'Max_Salary']] = df[['Min_Salary', 'Max_Salary']].astype('int')
df[['Min_Salary', 'Max_Salary']].dtypes

Min_Salary    int32
Max_Salary    int32
dtype: object

In [9]:
df['Avg_Salary'] = df[['Min_Salary', 'Max_Salary']].mean(axis=1).astype('int')

#### Droping duplicates

In [10]:
df[df.duplicated(subset=['Job_Title', 'Job_Description', 'Company_Name', 'Industry', 'Sector', 'Min_Salary', 'Max_Salary']) == 1]
df = df.drop_duplicates(subset=['Job_Title', 'Job_Description', 'Company_Name', 'Industry', 'Sector', 'Min_Salary', 'Max_Salary'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 659 entries, 0 to 671
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              659 non-null    int64  
 1   Job_Title          659 non-null    object 
 2   Salary Estimate    659 non-null    object 
 3   Job_Description    659 non-null    object 
 4   Rating             659 non-null    float64
 5   Company_Name       659 non-null    object 
 6   Location           659 non-null    object 
 7   Headquarters       659 non-null    object 
 8   Size               659 non-null    object 
 9   Founded            659 non-null    int64  
 10  Type_of_ownership  659 non-null    object 
 11  Industry           659 non-null    object 
 12  Sector             659 non-null    object 
 13  Revenue            659 non-null    object 
 14  Competitors        659 non-null    object 
 15  Min_Salary         659 non-null    int32  
 16  Max_Salary         659 non-null

#### Clean and split column __Location__

In [11]:
df[['City', 'State', 'Else']] = df['Location'].str.split(',', expand=True)
df.head(2)

,index,Job_Title,Salary Estimate,Job_Description,Rating,Company_Name,Location,Headquarters,Size,Founded,...,Industry,Sector,Revenue,Competitors,Min_Salary,Max_Salary,Avg_Salary,City,State,Else
0,0,Sr Data Scientist,137000-171000 (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,...,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna",137000,171000,154000,New York,NY,None
1,1,Data Scientist,137000-171000 (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,...,Research & Development,Business Services,$1 to $2 billion (USD),-1,137000,171000,154000,Chantilly,VA,None


In [12]:
row_problem = df['Else'] == ' MD'
df[row_problem]

,index,Job_Title,Salary Estimate,Job_Description,Rating,Company_Name,Location,Headquarters,Size,Founded,...,Industry,Sector,Revenue,Competitors,Min_Salary,Max_Salary,Avg_Salary,City,State,Else
534,534,Data Scientist,66000-112000 (Glassdoor est.),Job Description\nMUST HAVE SECRET CLEARANCE\n\...,3.4,"MILVETS Systems Technology, Inc.\n3.4","Patuxent, Anne Arundel, MD","Orlando, FL",51 to 200 employees,1986,...,IT Services,Information Technology,$25 to $50 million (USD),-1,66000,112000,89000,Patuxent,Anne Arundel,MD


In [13]:
df.loc[534, ['State']] = 'MD'
df.loc[534]

index                                                              534
Job_Title                                               Data Scientist
Salary Estimate                          66000-112000 (Glassdoor est.)
Job_Description      Job Description\nMUST HAVE SECRET CLEARANCE\n\...
Rating                                                             3.4
Company_Name                     MILVETS Systems Technology, Inc.\n3.4
Location                                    Patuxent, Anne Arundel, MD
Headquarters                                               Orlando, FL
Size                                               51 to 200 employees
Founded                                                           1986
Type_of_ownership                                    Company - Private
Industry                                                   IT Services
Sector                                          Information Technology
Revenue                                       $25 to $50 million (USD)
Compet

In [14]:
df = df.drop('Else', axis=1)
df.columns

Index(['index', 'Job_Title', 'Salary Estimate', 'Job_Description', 'Rating',
       'Company_Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type_of_ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'Min_Salary', 'Max_Salary', 'Avg_Salary', 'City', 'State'],
      dtype='object')

#### Check __Index__ for values beside float

In [15]:
df['index'] = df['index'].astype(float)
df['index'].dtype

dtype('float64')

##### If I was able to change datatype to float64 without any error that means this column has only numerical value.

#### Column __Rating__ looking for different values and fix values that are lower than 1 to 0

In [16]:
df['Rating'].unique()

array([ 3.1,  4.2,  3.8,  3.5,  2.9,  3.9,  4.4,  3.6,  4.5,  4.7,  3.7,
        3.4,  4.1,  3.2,  4.3,  2.8,  5. ,  4.8,  3.3,  2.7,  2.2,  2.6,
        4. ,  2.5,  4.9,  2.4, -1. ,  2.3,  4.6,  3. ,  2.1,  2. ])

In [17]:
df[df['Rating'] < 1.]
filt = df['Rating'] < 1.
df.loc[filt, ['Rating']] = 0

#### __Company_Name__ column, cleaning rightside of the every row that has unnecassary information 

In [18]:
filt = df['Company_Name'].str.contains('\n', regex=False)
df.loc[filt, 'Company_Name'] = df.loc[filt, 'Company_Name'].str[:-4]
df['Company_Name'].str.contains('\n', regex=False).unique()

array([False])

#### Cleaning column __Headquarters__

In [19]:
df['Headquarters'].str.contains('\/.!?').unique()

array([False])

#### Cleaning column __Size__

In [20]:
filt = df['Size'] == '-1'
df.loc[filt, 'Size'] = 'Unknown'
df['Size'].unique()

array(['1001 to 5000 employees', '5001 to 10000 employees',
       '501 to 1000 employees', '51 to 200 employees', '10000+ employees',
       '201 to 500 employees', '1 to 50 employees', 'Unknown'],
      dtype=object)

#### Cleaning column __Type_of_ownership__


In [21]:
filt = df['Type_of_ownership'] == '-1'
df.loc[filt, 'Type_of_ownership'] = 'Unknown'
df['Type_of_ownership'].unique()

array(['Nonprofit Organization', 'Company - Public',
       'Private Practice / Firm', 'Company - Private', 'Government',
       'Subsidiary or Business Segment', 'Other Organization', 'Unknown',
       'Hospital', 'Self-employed', 'College / University', 'Contract'],
      dtype=object)

#### Clean column __Industry__

In [22]:
filt = df['Industry'] == '-1'
df.loc[filt, 'Industry'] = 'Unknown'
df['Industry'].unique()

array(['Insurance Carriers', 'Research & Development', 'Consulting',
       'Electrical & Electronic Manufacturing', 'Advertising & Marketing',
       'Computer Hardware & Software', 'Biotech & Pharmaceuticals',
       'Consumer Electronics & Appliances Stores',
       'Enterprise Software & Network Solutions', 'IT Services', 'Energy',
       'Chemical Manufacturing', 'Federal Agencies', 'Internet',
       'Health Care Services & Hospitals',
       'Investment Banking & Asset Management', 'Aerospace & Defense',
       'Utilities', 'Unknown', 'Express Delivery Services',
       'Staffing & Outsourcing', 'Insurance Agencies & Brokerages',
       'Consumer Products Manufacturing', 'Industrial Manufacturing',
       'Food & Beverage Manufacturing', 'Banks & Credit Unions',
       'Video Games', 'Shipping', 'Telecommunications Services',
       'Lending', 'Cable, Internet & Telephone Providers', 'Real Estate',
       'Venture Capital & Private Equity', 'Miscellaneous Manufacturing',
       

#### Cleaning column __Sector__

In [23]:
filt = df['Sector'] == '-1'
df.loc[filt, 'Sector'] = 'Unknown'
df['Sector'].unique()

array(['Insurance', 'Business Services', 'Manufacturing',
       'Information Technology', 'Biotech & Pharmaceuticals', 'Retail',
       'Oil, Gas, Energy & Utilities', 'Government', 'Health Care',
       'Finance', 'Aerospace & Defense', 'Unknown',
       'Transportation & Logistics', 'Media', 'Telecommunications',
       'Real Estate', 'Travel & Tourism', 'Agriculture & Forestry',
       'Education', 'Accounting & Legal', 'Non-Profit',
       'Construction, Repair & Maintenance', 'Consumer Services'],
      dtype=object)

#### Cleaning column __Revenue__

In [24]:
filt = df['Revenue'] == '-1'
df.loc[filt, 'Revenue'] = 'Unknown / Non-Applicable'
df['Revenue'].unique()

array(['Unknown / Non-Applicable', '$1 to $2 billion (USD)',
       '$100 to $500 million (USD)', '$10+ billion (USD)',
       '$2 to $5 billion (USD)', '$500 million to $1 billion (USD)',
       '$5 to $10 billion (USD)', '$10 to $25 million (USD)',
       '$25 to $50 million (USD)', '$50 to $100 million (USD)',
       '$1 to $5 million (USD)', '$5 to $10 million (USD)',
       'Less than $1 million (USD)'], dtype=object)

#### Cleaning column __Competitors__

In [25]:
filt = df['Competitors'] == '-1'
df.loc[filt, 'Competitors'] = 'Unknown'
df['Competitors'].unique()

array(['EmblemHealth, UnitedHealth Group, Aetna', 'Unknown',
       'MKS Instruments, Pfeiffer Vacuum, Agilent Technologies',
       'Commerce Signals, Cardlytics, Yodlee',
       'Square, PayPal, H&R Block',
       'Leidos, CACI International, Booz Allen Hamilton',
       'Slalom, Daugherty Business Solutions',
       'Oak Ridge National Laboratory, National Renewable Energy Lab, Los Alamos National Laboratory',
       'CDW, PCM, SHI International',
       'Crossix Solutions Inc., AppNexus, The Trade Desk',
       'Northwestern Mutual', 'Puppet, Ansible, SaltStack',
       'Enlivant, Sunrise Senior Living, Brookdale Senior Living',
       'TrueCar, Cars.com, Kelley Blue Book',
       'Travelers, Allstate, State Farm', 'Novartis, Baxter, Pfizer',
       'Skyhigh Networks, Zscaler, NortonLifeLock',
       'Facebook, Google, Pinterest', 'DoorDash, Uber, Grubhub',
       'Munich Re, Hannover RE, SCOR', "IMAGE Skincare, Aveda, Kiehl's",
       'Luxoft, EPAM, Capgemini Invent', 'Sequenom',


#### Deleting __Salary Estimate__, __Location__ and __index__ columns

In [26]:
del df['Salary Estimate']
del df['Location']
del df['index']

#### Cleaning column __Max_Salary__

In [27]:
#filt = df['Max_Salary'].str.contains('Employer est.', regex=False)
#df.loc[filt, 'Max_Salary'] = df.loc[filt, 'Max_Salary'].str[:8]
#df['Max_Salary'].unique()

In [28]:
#filt = df['Max_Salary'].str.contains('Glassdoor est.', regex=False)
#df.loc[filt, 'Max_Salary'] = df.loc[filt, 'Max_Salary'].str[:-17]
#df['Max_Salary'].unique()

#### Creating a column which will have job name (data analyst, data engineer or data scientist) if this job name is in __Job_Title__

In [29]:
key_words = ['data scientist', 'data engineer', 'data analyst']

def find_matching_words(row):
    words = key_words
    words_row = row['Job_Title'].lower()
    matched_words = [word for word in words if word in words_row]
    if matched_words:
        return " ".join(matched_words)
    else:
        return np.nan
        
df['Title'] = df.apply(lambda row: find_matching_words(row), axis=1)

Now I will remove all the rows that contain nan value in 'Title'

In [30]:
df = df.dropna(subset=['Title'], how='all')

#### Creating column __seniority__

In [31]:
def seniority(row):
    senior = 'senior'
    job_title = row['Job_Title'].lower()
    if senior in job_title:
        return senior
    else:
        return np.nan
        
df['seniority'] = df.apply(lambda row: seniority(row), axis=1)

#### Creating columns for every key skill if it is in column __Job_Description__

In [32]:
key_skills = ['python', 'sql', 'excel', 'tableau', 'power bi', 'aws', 'spark', 'azure']

def skill_find(row, index):
    desc = row['Job_Description'].lower()
    if key_skills[index] in desc:
        return key_skills[index]
    else:
        return np.nan
for index, key_skill_column in enumerate(key_skills):
    df[key_skill_column] = df.apply(lambda row: skill_find(row, index), axis=1)


#### New order for columns

In [33]:
df.columns

Index(['Job_Title', 'Job_Description', 'Rating', 'Company_Name',
       'Headquarters', 'Size', 'Founded', 'Type_of_ownership', 'Industry',
       'Sector', 'Revenue', 'Competitors', 'Min_Salary', 'Max_Salary',
       'Avg_Salary', 'City', 'State', 'Title', 'seniority', 'python', 'sql',
       'excel', 'tableau', 'power bi', 'aws', 'spark', 'azure'],
      dtype='object')

In [34]:
new_order = ['Job_Title', 'Min_Salary', 'Max_Salary', 'Avg_Salary', 'Job_Description', 'Rating',
       'Company_Name', 'City', 'State', 'Headquarters', 'Size', 'Founded',
       'Type_of_ownership', 'Industry', 'Sector', 'Revenue', 'Competitors', 'Title', 'seniority', 'python', 'sql',
       'excel', 'tableau', 'power bi', 'aws', 'spark', 'azure']
df = df[new_order]
df.head(2)

,Job_Title,Min_Salary,Max_Salary,Avg_Salary,Job_Description,Rating,Company_Name,City,State,Headquarters,...,Title,seniority,python,sql,excel,tableau,power bi,aws,spark,azure
0,Sr Data Scientist,137000,171000,154000,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,New York,NY,"New York, NY",...,data scientist,NaN,NaN,NaN,NaN,NaN,NaN,aws,NaN,azure
1,Data Scientist,137000,171000,154000,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,Chantilly,VA,"Herndon, VA",...,data scientist,NaN,NaN,sql,NaN,NaN,NaN,NaN,NaN,NaN


#### Write clean and modified data to new csv file


In [35]:
df.to_csv('C:/Users/ADMIN/Folder with projects/Cleanded_DS.csv')